### Data Clearing

In [1]:
import requests as r
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import time
import random
import math 
import os
import re


pd.options.display.max_rows = None
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings("ignore")

### 0) Usunięcie konkretnych danych
### 1)team_score 
### 2) Stworzenie zmiennych mv_change, individual stats change, team_score_change
### 3) Zapisanie ramki danych

In [2]:
dataframe_raw = pd.read_csv('dataframe_raw.csv').drop(columns=['Unnamed: 0'])

print(dataframe_raw.shape)
dataframe_raw = dataframe_raw[(dataframe_raw['mv'] != 0) & (dataframe_raw['next_season_mv'] != 0)] # odejmuje to ok. 200 rekordow
print(dataframe_raw.shape)
dataframe_raw = dataframe_raw[dataframe_raw['Min'] >= 90] # odejmuje to ok. 300 rekordow
print(dataframe_raw.shape)

dataframe_raw.drop(columns=['Save%_gk', 'Cmp%_gk', 'OPA/90_gk'], inplace=True)

dataframe_raw.head()

(2048, 32)
(1805, 32)
(1517, 32)


,Player,tm_code,position,season,mv,club,last_season_before_transfer,next_season_mv,Age,Starts,Min,Gls,Ast,CrdY,CrdR,SoT,G/Sh,passing_Att,passing_Cmp%,TklW,Blocks,Int,Clr,dribbles_Att,Carries,Rec,league_place,cl_place,cup_place
0,Stephan Lichtsteiner,2865,defence,2018,1500000.0,Arsenal FC,1,800000.0,34.0,19.0,1731.0,1.0,1.0,5.0,0.0,1.0,0.11,930.0,77.1,13.0,17.0,15.0,35.0,8.0,515.0,697.0,5,0.0,8
1,James Milner,3333,midfield,2018,15000000.0,Liverpool FC,0,6500000.0,32.0,31.0,2837.0,7.0,6.0,9.0,1.0,8.0,0.06,2262.0,79.3,37.0,55.0,28.0,43.0,27.0,1562.0,1722.0,2,1.0,32
2,James Milner,3333,midfield,2019,6500000.0,Liverpool FC,0,3000000.0,33.0,17.0,1656.0,4.0,4.0,6.0,0.0,6.0,0.06,1223.0,84.1,25.0,17.0,13.0,18.0,18.0,933.0,989.0,1,16.0,8
3,James Milner,3333,midfield,2020,3000000.0,Liverpool FC,0,2000000.0,34.0,19.0,1760.0,0.0,1.0,6.0,0.0,4.0,0.00,1178.0,82.3,33.0,37.0,22.0,20.0,18.0,801.0,939.0,3,8.0,16
4,James Milner,3333,midfield,2021,2000000.0,Liverpool FC,0,1500000.0,35.0,14.0,1387.0,0.0,3.0,5.0,0.0,3.0,0.00,949.0,81.6,24.0,23.0,22.0,23.0,7.0,606.0,770.0,2,2.0,1


### club_score

In [3]:
dataframe_raw['club_score'] = dataframe_raw['league_place'].apply(lambda x: 1/x * 4.5 if x != 0 else 0) + dataframe_raw['cl_place'].apply(lambda x: 1/x * 4.5 if x != 0 else 0) + dataframe_raw['cup_place'].apply(lambda x: 1/x * 1 if x != 0 else 0)

dataframe_raw.drop(columns=['league_place', 'cl_place', 'cup_place'], inplace=True)

dataframe_raw.head()

,Player,tm_code,position,season,mv,club,last_season_before_transfer,next_season_mv,Age,Starts,Min,Gls,Ast,CrdY,CrdR,SoT,G/Sh,passing_Att,passing_Cmp%,TklW,Blocks,Int,Clr,dribbles_Att,Carries,Rec,club_score
0,Stephan Lichtsteiner,2865,defence,2018,1500000.0,Arsenal FC,1,800000.0,34.0,19.0,1731.0,1.0,1.0,5.0,0.0,1.0,0.11,930.0,77.1,13.0,17.0,15.0,35.0,8.0,515.0,697.0,1.02500
1,James Milner,3333,midfield,2018,15000000.0,Liverpool FC,0,6500000.0,32.0,31.0,2837.0,7.0,6.0,9.0,1.0,8.0,0.06,2262.0,79.3,37.0,55.0,28.0,43.0,27.0,1562.0,1722.0,6.78125
2,James Milner,3333,midfield,2019,6500000.0,Liverpool FC,0,3000000.0,33.0,17.0,1656.0,4.0,4.0,6.0,0.0,6.0,0.06,1223.0,84.1,25.0,17.0,13.0,18.0,18.0,933.0,989.0,4.90625
3,James Milner,3333,midfield,2020,3000000.0,Liverpool FC,0,2000000.0,34.0,19.0,1760.0,0.0,1.0,6.0,0.0,4.0,0.00,1178.0,82.3,33.0,37.0,22.0,20.0,18.0,801.0,939.0,2.12500
4,James Milner,3333,midfield,2021,2000000.0,Liverpool FC,0,1500000.0,35.0,14.0,1387.0,0.0,3.0,5.0,0.0,3.0,0.00,949.0,81.6,24.0,23.0,22.0,23.0,7.0,606.0,770.0,5.50000


### individual stats change, club_score_change

In [4]:
dataframe_raw.reset_index(drop=True, inplace=True)

In [5]:
dataframe_raw = dataframe_raw.sort_values(['tm_code', 'season'])

columns_to_iterate = [
    'Starts', 'Min', 'Gls', 'Ast', 'CrdY', 'CrdR', 'SoT', 'G/Sh', 'passing_Att', 'passing_Cmp%', 'TklW', 
    'Blocks', 'Int', 'Clr', 'dribbles_Att', 'Carries', 'Rec', 'club_score'
]

for col in columns_to_iterate:
    dataframe_raw[f'{col}_change'] = (dataframe_raw['tm_code'] == dataframe_raw['tm_code'].shift(1)) & (dataframe_raw['season']-1 == dataframe_raw['season'].shift(1))
    dataframe_raw[f'{col}_change'] = dataframe_raw[f'{col}_change'].apply(lambda x: 1 if x else pd.np.nan)
    
    # zmiana absolutna
    dataframe_raw[f'{col}_abs_change'] = dataframe_raw[f'{col}_change'] * (dataframe_raw[col] - dataframe_raw[col].shift(1))
    
    # roznica logarytmów
    dataframe_raw[col] = dataframe_raw[col].apply(lambda x: np.log(x+1))
    dataframe_raw[f'{col}_log_change'] = dataframe_raw[f'{col}_change'] * (dataframe_raw[col] - dataframe_raw[col].shift(1))
    
    dataframe_raw.drop(columns=[f'{col}_change'], inplace=True)
    

    


dataframe_raw = dataframe_raw.drop(columns=columns_to_iterate)


print(dataframe_raw.shape)
dataframe_raw = dataframe_raw.dropna()
print(dataframe_raw.shape)

dataframe_raw.head(10)

(1517, 45)
(873, 45)


,Player,tm_code,position,season,mv,club,last_season_before_transfer,next_season_mv,Age,Starts_abs_change,Starts_log_change,Min_abs_change,Min_log_change,Gls_abs_change,Gls_log_change,Ast_abs_change,Ast_log_change,CrdY_abs_change,CrdY_log_change,CrdR_abs_change,CrdR_log_change,SoT_abs_change,SoT_log_change,G/Sh_abs_change,G/Sh_log_change,passing_Att_abs_change,passing_Att_log_change,passing_Cmp%_abs_change,passing_Cmp%_log_change,TklW_abs_change,TklW_log_change,Blocks_abs_change,Blocks_log_change,Int_abs_change,Int_log_change,Clr_abs_change,Clr_log_change,dribbles_Att_abs_change,dribbles_Att_log_change,Carries_abs_change,Carries_log_change,Rec_abs_change,Rec_log_change,club_score_abs_change,club_score_log_change
2,James Milner,3333,midfield,2019,6500000.0,Liverpool FC,0,3000000.0,33.0,-14.0,-0.575364,-1181.0,-0.538091,-3.0,-0.470004,-2.0,-0.336472,-3.0,-0.356675,-1.0,-0.693147,-2.0,-0.251314,0.00,0.000000,-1039.0,-0.614567,4.8,0.058057,-12.0,-0.379490,-38.0,-1.134980,-15.0,-0.728239,-25.0,-0.839751,-9.0,-0.387766,-629.0,-0.514886,-733.0,-0.554117,-1.875000,-0.275706
3,James Milner,3333,midfield,2020,3000000.0,Liverpool FC,0,2000000.0,34.0,2.0,0.105361,104.0,0.060873,-4.0,-1.609438,-3.0,-0.916291,0.0,0.000000,0.0,0.000000,-2.0,-0.336472,-0.06,-0.058269,-45.0,-0.037458,-1.8,-0.021378,8.0,0.268264,20.0,0.747214,9.0,0.496437,2.0,0.100083,0.0,0.000000,-132.0,-0.152368,-50.0,-0.051825,-2.781250,-0.636577
4,James Milner,3333,midfield,2021,2000000.0,Liverpool FC,0,1500000.0,35.0,-5.0,-0.287682,-373.0,-0.238018,0.0,0.000000,2.0,0.693147,-1.0,-0.154151,0.0,0.000000,-1.0,-0.223144,0.00,0.000000,-229.0,-0.215960,-0.7,-0.008439,-9.0,-0.307485,-14.0,-0.459532,0.0,0.000000,3.0,0.133531,-11.0,-0.864997,-195.0,-0.278580,-169.0,-0.198192,3.375000,0.732368
5,James Milner,3333,midfield,2022,1500000.0,Liverpool FC,1,1000000.0,36.0,-2.0,-0.143101,-81.0,-0.060129,0.0,0.000000,-1.0,-0.287682,-2.0,-0.405465,0.0,0.000000,-2.0,-0.693147,0.00,0.000000,-69.0,-0.075404,-3.5,-0.043297,3.0,0.113329,-4.0,-0.182322,-8.0,-0.427444,-6.0,-0.287682,18.0,1.178655,-64.0,-0.111419,-86.0,-0.118270,-4.256250,-1.063654
8,Cristiano Ronaldo,8198,attack,2022,15000000.0,Manchester United,1,15000000.0,37.0,-25.0,-1.185624,-2131.0,-1.104592,-21.0,-1.832581,-1.0,-0.287682,-6.0,-0.916291,0.0,0.000000,-42.0,-1.442384,-0.11,-0.100541,-773.0,-1.183830,-1.9,-0.023357,-5.0,-0.810930,-13.0,-2.014903,-1.0,-0.405465,-26.0,-1.213023,-27.0,-0.826679,-526.0,-1.154852,-800.0,-1.134886,1.437500,0.528844
15,Ashley Young,14086,defence,2019,3200000.0,Manchester United,1,600000.0,34.0,-24.0,-0.955511,-2224.0,-0.989053,-1.0,-0.405465,-2.0,-0.693147,-5.0,-0.441833,-1.0,-0.693147,-2.0,-0.693147,0.03,0.025975,-1387.0,-0.942828,4.9,0.064624,-32.0,-1.241713,-30.0,-1.315677,-49.0,-1.314321,-59.0,-1.045124,-17.0,-0.535518,-752.0,-0.963438,-839.0,-0.878679,0.406250,0.159849
20,Hugo Lloris,17965,goalkeeper,2019,14500000.0,Tottenham Hotspur,0,9000000.0,32.0,-17.0,-0.474458,-1601.0,-0.519898,0.0,0.000000,0.0,0.000000,0.0,0.000000,-1.0,-0.693147,0.0,0.000000,0.00,0.000000,-931.0,-0.876347,-3.6,-0.047315,0.0,0.000000,0.0,0.000000,1.0,0.693147,-9.0,-0.826679,-1.0,-0.693147,-591.0,-0.951160,-604.0,-1.214505,-2.562500,-0.807558
21,Hugo Lloris,17965,goalkeeper,2020,9000000.0,Tottenham Hotspur,0,7000000.0,33.0,20.0,0.538997,1943.0,0.602956,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,517.0,0.575835,0.3,0.004030,1.0,0.693147,0.0,0.000000,-1.0,-0.693147,4.0,0.451985,0.0,0.000000,234.0,0.487986,210.0,0.600774,0.080357,0.038221
22,Hugo Lloris,17965,goalkeeper,2021,7000000.0,Tottenham Hotspur,0,3000000.0,34.0,-4.0,-0.087011,-390.0,-0.095287,0.0,0.000000,0.0,0.000000,2.0,1.098612,0.0,0.000000,0.0,0.000000,0.00,0.000000,-193.0,-0.178434,6.3,0.081073,1.0,0.405465,0.0,0.000000,1.0,0.693147,14.0,0.820981,0.0,0.000000,-47.0,-0.080731,-76.0,-0.178458,0.232143,0.102857
23,Hugo Lloris,17965,goalkeeper,2022,3000000.0,Tottenham Hotspur,1,1500000.0,35.0,-12.0,-0.318454,-1155.0,-0.351088,0.0,0.0000

### mv_change

In [6]:
# zmiana absolutna
dataframe_raw['mv_abs_change'] = dataframe_raw['next_season_mv'] - dataframe_raw['mv']

# wejscie na logarytmy
dataframe_raw['mv'] = dataframe_raw['mv'].apply(lambda x: np.log(x))
dataframe_raw['next_season_mv'] = dataframe_raw['next_season_mv'].apply(lambda x: np.log(x))
dataframe_raw['mv_log_change'] = dataframe_raw['next_season_mv'] - dataframe_raw['mv']


dataframe_raw = dataframe_raw.drop(columns=['mv'])
dataframe_raw = dataframe_raw.drop(columns=['next_season_mv'])

dataframe_raw = dataframe_raw.drop(columns=['Player', 'tm_code'])

print(dataframe_raw.shape)
dataframe_raw.head(10)

(873, 43)


,position,season,club,last_season_before_transfer,Age,Starts_abs_change,Starts_log_change,Min_abs_change,Min_log_change,Gls_abs_change,Gls_log_change,Ast_abs_change,Ast_log_change,CrdY_abs_change,CrdY_log_change,CrdR_abs_change,CrdR_log_change,SoT_abs_change,SoT_log_change,G/Sh_abs_change,G/Sh_log_change,passing_Att_abs_change,passing_Att_log_change,passing_Cmp%_abs_change,passing_Cmp%_log_change,TklW_abs_change,TklW_log_change,Blocks_abs_change,Blocks_log_change,Int_abs_change,Int_log_change,Clr_abs_change,Clr_log_change,dribbles_Att_abs_change,dribbles_Att_log_change,Carries_abs_change,Carries_log_change,Rec_abs_change,Rec_log_change,club_score_abs_change,club_score_log_change,mv_abs_change,mv_log_change
2,midfield,2019,Liverpool FC,0,33.0,-14.0,-0.575364,-1181.0,-0.538091,-3.0,-0.470004,-2.0,-0.336472,-3.0,-0.356675,-1.0,-0.693147,-2.0,-0.251314,0.00,0.000000,-1039.0,-0.614567,4.8,0.058057,-12.0,-0.379490,-38.0,-1.134980,-15.0,-0.728239,-25.0,-0.839751,-9.0,-0.387766,-629.0,-0.514886,-733.0,-0.554117,-1.875000,-0.275706,-3500000.0,-0.773190
3,midfield,2020,Liverpool FC,0,34.0,2.0,0.105361,104.0,0.060873,-4.0,-1.609438,-3.0,-0.916291,0.0,0.000000,0.0,0.000000,-2.0,-0.336472,-0.06,-0.058269,-45.0,-0.037458,-1.8,-0.021378,8.0,0.268264,20.0,0.747214,9.0,0.496437,2.0,0.100083,0.0,0.000000,-132.0,-0.152368,-50.0,-0.051825,-2.781250,-0.636577,-1000000.0,-0.405465
4,midfield,2021,Liverpool FC,0,35.0,-5.0,-0.287682,-373.0,-0.238018,0.0,0.000000,2.0,0.693147,-1.0,-0.154151,0.0,0.000000,-1.0,-0.223144,0.00,0.000000,-229.0,-0.215960,-0.7,-0.008439,-9.0,-0.307485,-14.0,-0.459532,0.0,0.000000,3.0,0.133531,-11.0,-0.864997,-195.0,-0.278580,-169.0,-0.198192,3.375000,0.732368,-500000.0,-0.287682
5,midfield,2022,Liverpool FC,1,36.0,-2.0,-0.143101,-81.0,-0.060129,0.0,0.000000,-1.0,-0.287682,-2.0,-0.405465,0.0,0.000000,-2.0,-0.693147,0.00,0.000000,-69.0,-0.075404,-3.5,-0.043297,3.0,0.113329,-4.0,-0.182322,-8.0,-0.427444,-6.0,-0.287682,18.0,1.178655,-64.0,-0.111419,-86.0,-0.118270,-4.256250,-1.063654,-500000.0,-0.405465
8,attack,2022,Manchester United,1,37.0,-25.0,-1.185624,-2131.0,-1.104592,-21.0,-1.832581,-1.0,-0.287682,-6.0,-0.916291,0.0,0.000000,-42.0,-1.442384,-0.11,-0.100541,-773.0,-1.183830,-1.9,-0.023357,-5.0,-0.810930,-13.0,-2.014903,-1.0,-0.405465,-26.0,-1.213023,-27.0,-0.826679,-526.0,-1.154852,-800.0,-1.134886,1.437500,0.528844,0.0,0.000000
15,defence,2019,Manchester United,1,34.0,-24.0,-0.955511,-2224.0,-0.989053,-1.0,-0.405465,-2.0,-0.693147,-5.0,-0.441833,-1.0,-0.693147,-2.0,-0.693147,0.03,0.025975,-1387.0,-0.942828,4.9,0.064624,-32.0,-1.241713,-30.0,-1.315677,-49.0,-1.314321,-59.0,-1.045124,-17.0,-0.535518,-752.0,-0.963438,-839.0,-0.878679,0.406250,0.159849,-2600000.0,-1.673976
20,goalkeeper,2019,Tottenham Hotspur,0,32.0,-17.0,-0.474458,-1601.0,-0.519898,0.0,0.000000,0.0,0.000000,0.0,0.000000,-1.0,-0.693147,0.0,0.000000,0.00,0.000000,-931.0,-0.876347,-3.6,-0.047315,0.0,0.000000,0.0,0.000000,1.0,0.693147,-9.0,-0.826679,-1.0,-0.693147,-591.0,-0.951160,-604.0,-1.214505,-2.562500,-0.807558,-5500000.0,-0.476924
21,goalkeeper,2020,Tottenham Hotspur,0,33.0,20.0,0.538997,1943.0,0.602956,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,517.0,0.575835,0.3,0.004030,1.0,0.693147,0.0,0.000000,-1.0,-0.693147,4.0,0.451985,0.0,0.000000,234.0,0.487986,210.0,0.600774,0.080357,0.038221,-2000000.0,-0.251314
22,goalkeeper,2021,Tottenham Hotspur,0,34.0,-4.0,-0.087011,-390.0,-0.095287,0.0,0.000000,0.0,0.000000,2.0,1.098612,0.0,0.000000,0.0,0.000000,0.00,0.000000,-193.0,-0.178434,6.3,0.081073,1.0,0.405465,0.0,0.000000,1.0,0.693147,14.0,0.820981,0.0,0.000000,-47.0,-0.080731,-76.0,-0.178458,0.232143,0.102857,-4000000.0,-0.847298
23,goalkeeper,2022,Tottenham Hotspur,1,35.0,-12.0,-0.318454,-1155.0,-0.351088,0.0,0.000000,0.0,0.000000,-2.0,-1.098612,0.0,0.000000,0.0,0.000000,0.00,0.000000,-83.0,-0.087748,-1.2,-0.014944,-1.0,-0.405465,0.0,0.000000,1.0,0.405465,-9.0,-0.446287,0.0,0.000000,-6.0,-0.010791,52.0,0.125466,-0.500000,-0.2

### dummy variables, final DF

In [9]:
dataframe_linear_reg = dataframe_raw.copy()

dataframe_linear_reg = pd.concat([dataframe_linear_reg, pd.get_dummies(dataframe_linear_reg['club'])], axis=1)

dataframe_linear_reg = pd.concat([dataframe_linear_reg, pd.get_dummies(dataframe_linear_reg['position'])], axis=1)

dataframe_linear_reg = pd.concat([dataframe_linear_reg, pd.get_dummies(dataframe_linear_reg['season'])], axis=1)

dataframe_linear_reg.drop(columns=['position', 'season', 'club'], inplace=True)

dataframe_linear_reg.reset_index(drop=True, inplace=True)


dataframe_linear_reg.to_csv('df_final.csv')

print(dataframe_linear_reg.shape)
dataframe_linear_reg.head(10)

(873, 60)


,last_season_before_transfer,Age,Starts_abs_change,Starts_log_change,Min_abs_change,Min_log_change,Gls_abs_change,Gls_log_change,Ast_abs_change,Ast_log_change,CrdY_abs_change,CrdY_log_change,CrdR_abs_change,CrdR_log_change,SoT_abs_change,SoT_log_change,G/Sh_abs_change,G/Sh_log_change,passing_Att_abs_change,passing_Att_log_change,passing_Cmp%_abs_change,passing_Cmp%_log_change,TklW_abs_change,TklW_log_change,Blocks_abs_change,Blocks_log_change,Int_abs_change,Int_log_change,Clr_abs_change,Clr_log_change,dribbles_Att_abs_change,dribbles_Att_log_change,Carries_abs_change,Carries_log_change,Rec_abs_change,Rec_log_change,club_score_abs_change,club_score_log_change,mv_abs_change,mv_log_change,Arsenal FC,Brighton & Hove Albion,Chelsea FC,Crystal Palace,Everton FC,Liverpool FC,Manchester City,Manchester United,Newcastle United,Tottenham Hotspur,West Ham United,Wolverhampton Wanderers,attack,defence,goalkeeper,midfield,2019,2020,2021,2022
0,0,33.0,-14.0,-0.575364,-1181.0,-0.538091,-3.0,-0.470004,-2.0,-0.336472,-3.0,-0.356675,-1.0,-0.693147,-2.0,-0.251314,0.00,0.000000,-1039.0,-0.614567,4.8,0.058057,-12.0,-0.379490,-38.0,-1.134980,-15.0,-0.728239,-25.0,-0.839751,-9.0,-0.387766,-629.0,-0.514886,-733.0,-0.554117,-1.875000,-0.275706,-3500000.0,-0.773190,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0
1,0,34.0,2.0,0.105361,104.0,0.060873,-4.0,-1.609438,-3.0,-0.916291,0.0,0.000000,0.0,0.000000,-2.0,-0.336472,-0.06,-0.058269,-45.0,-0.037458,-1.8,-0.021378,8.0,0.268264,20.0,0.747214,9.0,0.496437,2.0,0.100083,0.0,0.000000,-132.0,-0.152368,-50.0,-0.051825,-2.781250,-0.636577,-1000000.0,-0.405465,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,0,35.0,-5.0,-0.287682,-373.0,-0.238018,0.0,0.000000,2.0,0.693147,-1.0,-0.154151,0.0,0.000000,-1.0,-0.223144,0.00,0.000000,-229.0,-0.215960,-0.7,-0.008439,-9.0,-0.307485,-14.0,-0.459532,0.0,0.000000,3.0,0.133531,-11.0,-0.864997,-195.0,-0.278580,-169.0,-0.198192,3.375000,0.732368,-500000.0,-0.287682,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,1,36.0,-2.0,-0.143101,-81.0,-0.060129,0.0,0.000000,-1.0,-0.287682,-2.0,-0.405465,0.0,0.000000,-2.0,-0.693147,0.00,0.000000,-69.0,-0.075404,-3.5,-0.043297,3.0,0.113329,-4.0,-0.182322,-8.0,-0.427444,-6.0,-0.287682,18.0,1.178655,-64.0,-0.111419,-86.0,-0.118270,-4.256250,-1.063654,-500000.0,-0.405465,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,1,37.0,-25.0,-1.185624,-2131.0,-1.104592,-21.0,-1.832581,-1.0,-0.287682,-6.0,-0.916291,0.0,0.000000,-42.0,-1.442384,-0.11,-0.100541,-773.0,-1.183830,-1.9,-0.023357,-5.0,-0.810930,-13.0,-2.014903,-1.0,-0.405465,-26.0,-1.213023,-27.0,-0.826679,-526.0,-1.154852,-800.0,-1.134886,1.437500,0.528844,0.0,0.000000,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
5,1,34.0,-24.0,-0.955511,-2224.0,-0.989053,-1.0,-0.405465,-2.0,-0.693147,-5.0,-0.441833,-1.0,-0.693147,-2.0,-0.693147,0.03,0.025975,-1387.0,-0.942828,4.9,0.064624,-32.0,-1.241713,-30.0,-1.315677,-49.0,-1.314321,-59.0,-1.045124,-17.0,-0.535518,-752.0,-0.963438,-839.0,-0.878679,0.406250,0.159849,-2600000.0,-1.673976,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
6,0,32.0,-17.0,-0.474458,-1601.0,-0.519898,0.0,0.000000,0.0,0.000000,0.0,0.000000,-1.0,-0.693147,0.0,0.000000,0.00,0.000000,-931.0,-0.876347,-3.6,-0.047315,0.0,0.000000,0.0,0.000000,1.0,0.693147,-9.0,-0.826679,-1.0,-0.693147,-591.0,-0.951160,-604.0,-1.214505,-2.562500,-0.807558,-5500000.0,-0.476924,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
7,0,33.0,20.0,0.538997,1943.0,0.602956,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,517.0,0.575835,0.3,0.004030,1.0,0.693147,0.0,0.000000,-1.0,-0.693147,4.0,0.451985,0.0,0.000000,234.0,0.487986,210.0,0.600774,0.080357,0.038221,-2000000.0,-0.251314,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
8,0,34.0,-4.0,-0.087011,-390.0,-0.095287,0.0,0.000000,0.0,0.000000,2.0,1.098612,0.0,0.000000,0.0,0.000000,0.00,0.000000,-193.0,-0.178434,6.3,0.081073,1.0,0.405465,0.0,0.000000,1.0,0.693147,14.0,0.820981,0.0,0.000000,-47.0,-0.080731,-76.0,-0.178458,0.232143,0.102857,-4000000.0,-0.847298,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,